*Supporting textbook chapters for week 8: Chapters 9.1 - 9.3* 

Topics:
* Classifying PDEs
* Elliptic equation solvers: Jacobi, Gauss-Seidel, overrelaxation
* Parabolic equation solver: Explicit FTCS (Forward Time, Centered Space)
* Parabolic and hyperbolic equation soliver: Implicit FTCS, Crank-Nicolson

# Classification and General Approach

* Solving partial differential equations is one of the pinnacles of computational physics, bringing together many methods.
* Each type comes with design decisions on how to discretize and implement numerical methods,
* Stability is crucial.
* Accuracy is crucial too.

Recall conical equations in geometry:
$$ \alpha x^2 + \beta xy + \gamma y^2 + \delta x + \varepsilon y = f, $$
classified using $ \Delta = \beta^2 - 4\alpha\gamma.$

1. $\Delta = 0$: equation for a parabola,
3. $\Delta < 0$: equation for an ellipse,
2. $\Delta > 0$: equation for a hyperbola.

Similar for PDEs:
$$ \alpha\frac{\partial^2 \phi}{\partial x^2} + \beta\frac{\partial^2 \phi}{\partial x\partial y} + \gamma\frac{\partial^2 \phi}{\partial y^2} + \delta \frac{\partial\phi}{\partial x} + \varepsilon\frac{\partial\phi}{\partial y} = f.$$


With $\Delta = \beta^2 - 4\alpha\gamma$,
1. $\Delta = 0$: parabolic PDE,
3. $\Delta < 0$: elliptic PDE,
2. $\Delta > 0$: hyperbolic PDE.

## Physics Examples

1. Canonical parabolic PDE: the diffusion equation, $ \kappa \frac{\partial ^2 T}{\partial x^2}  - \frac{\partial T}{\partial t} = 0$,
    $$x\to x,\quad y\to t,\quad \alpha \to \kappa,\quad \varepsilon \to - 1,\quad \beta, \gamma, \delta, f \to 0 \quad\Rightarrow\quad \beta^2 - 4\alpha\gamma = 0.$$
2. Canonical elliptic PDE: the Poisson equation, $\ \nabla^2 \phi = \rho$,
    $$x\to x,\quad y\to y,\quad\alpha, \gamma \to 1, f \to \rho, \beta, \delta, \varepsilon \to 0 \quad \Rightarrow \quad \beta^2 - 4\alpha\gamma = -4<0.$$
   * e.g. 2D electrostatics, with electric potential $\phi$ s.t. $\vec E  = \nabla \phi$, in the absence of charges $(\rho \equiv 0)$, have Gauss' law:
$\frac{\partial^2 \phi}{\partial x^2} + \frac{\partial^2 \phi}{\partial y^2} = 0$
3. Canonical hyperbolic PDE: the wave equation, $\ \displaystyle c^2 \frac{\partial ^2 \phi}{\partial x^2}  - \frac{\partial^2 \phi}{\partial t^2} = 0.$
    $$x\to x,\quad y\to t,\quad \alpha \to c^2,\quad \gamma \to - 1,\quad \beta, \delta, \varepsilon, f \to 0 \quad\Rightarrow\quad \beta^2 - 4\alpha\gamma = 4c^2 >0.$$

Note: we use these categorizations even when the spatial operator is $\nabla^2 = \partial_x^2 + \partial_y^2 + \partial_z^2$, i.e., for 4D PDEs.

## General approach

* Discretize system spatially and temporally: can use finite difference, spectral coefficients, etc.
* $\Rightarrow$ set of coupled ODEs that you need to solve in an efficient way.
* Spatial derivatives bring information in from neighbouring points $\Rightarrow$ coupling,
* $\Rightarrow$ errors depend on space and time and can get wave-like characteristics.
* For 2nd derivatives, recall central difference calculation (§5.10.5, p.197):
$$f''(x) = \frac{f(x+h) - 2f(x)+ f(x-h)}{h^2} - \frac{1}{12}h^2 f^{(4)}(x) + \dots{}$$

# Elliptic Equations

Start with simplest case of Gauss's Law with 2D Laplacian:
$$0 = \nabla^2 \phi  = \frac{\partial^2 \phi}{\partial x^2} + \frac{\partial^2 \phi}{\partial y^2},$$

On regular square grid of cell side length $a$, finite difference form is
\begin{align}
\frac{\partial^2 \phi}{\partial x^2} & \approx \frac{\phi(x+a, y) - 2\phi(x, y)+ \phi(x-a, y)}{a^2},\\
\frac{\partial^2 \phi}{\partial y^2} & \approx \frac{\phi(x, y+a) - 2\phi(x, y)+ \phi(x, y-a)}{a^2}.
\end{align}
Gauss's law then becomes:
$$0 \approx \phi(x+a, y) + \phi(x-a, y) + \phi(x, y+a) + \phi(x, y-a) - 4\phi(x, y)$$
at each location $(x, y)$.

* Put together a series of equations of the form
    $$\phi(x+a, y) + \phi(x-a, y) + \phi(x, y+a) + \phi(x, y-a) - 4\phi(x) = 0$$
    for each $x$ and $y$, subject to boundary conditions.
* $\phi$ or derivative $\partial\phi/\partial \xi$ ($\xi = x,\ y,$ or both) given on boundary.
 * If $\phi$ given, use this value for adjacent points.
 * If $\partial\phi/\partial \xi$ given, find algebraic relationship between points near to boundary using finite difference.
* Could solve using matrix methods $\mathbf L \phi = \mathbf R \phi$, but a simpler method is possible.


## Jacobi relaxation method

$$\phi(x+a, y) + \phi(x-a, y) + \phi(x, y+a) + \phi(x, y-a) - 4\phi(x) = 0$$
* Iterate the rule
$\phi_{new}(x, y) = \frac14\left[\phi(x+a, y) + \phi(x-a, y) + \phi(x, y+a) + \phi(x, y-a)\right].$
* Much like the relaxation method for finding solutions of $f(x) = x$,
* For this problem it turns out that Jacobi Relaxation is always stable and so always gives a solution!

## Overrelaxation method

$\phi_{new}(x,y) = $
$$(1+\omega)\left[\frac{\phi(x+a, y) + \phi(x-a, y) + \phi(x, y+a) + \phi(x, y-a)}4\right] - \omega \phi (x,y)$$
* When it works, it usually speeds up the calculation.
* Not always stable! How to choose $\omega$ is not always reproducible.
* see Newman's exercise 6.11 for a similar problem for finding $f(x)=x$.

## Gauss-Seidel method

* Replace function on the fly as in
$$\phi(x, y) \leftarrow \frac{\phi(x+a, y) + \phi(x-a, y) + \phi(x, y+a) + \phi(x, y-a)}4.$$
* Crucial difference: the LHS is $\phi$, not $\phi_{new}$: we use newer values as they are being computed (Jacobi used only old values to compute new one).
* This can be shown to run faster.
* Can be combined with overrelaxation.

# Parabolic PDEs

* Consider the 1D heat equation:
$$\frac{\partial T}{\partial t} = \kappa\frac{\partial^2 T}{\partial x^2},$$
* B.Cs.:
    $$T(x=0, t) = T_0, \quad T(x=L, t) = T_L.$$
* I.C.:
    $$T(x, t=0) = T_0 +  (T_L - T_0)\left(\frac{f(x) - f(0)}{f(L) - f(0)}\right)$$

## Explicit Forward Time Centred Space method

**Step 1: Discretize in space**

$\displaystyle x_m = \frac{m}{M}L = am, \quad m=0\dots{}M, \quad a = \frac{L}M$,

$T_m(t) = \left[T_0(t), \dots{}, T_{M}(t)\right]$

$\displaystyle \left.\frac{\partial^2 T}{\partial x^2}\right|_{x=x_m, t} \approx \frac{T_{m+1} - 2 T_m + T_{m-1}}{a^2}\quad$ ("centred space", CS)

**Step 2: Discretize in time**

$\displaystyle \frac{d T_m}{d t} \approx \kappa\frac{T_{m+1} - 2 T_m + T_{m-1}}{a^2},\quad m = 1 \dots{}, M-1$

Let $t_n = nh$, $h$ the time step. Let $T_m(t_n) \equiv T_m^n$.

$\displaystyle \Rightarrow \left.\frac{\partial T}{\partial t}\right|_{x=ma, t=nh} \approx \frac{T_{m}^{n+1} - T_m^n}{h} \equiv \kappa\frac{T_{m+1}^n - 2 T_m^n + T_{m-1}^n}{a^2}$ ("Forward Time", FT).

$\Rightarrow$ **Explicit FTCS method:**
$$\boxed{T_m^{n+1} = T_m^n + \frac{\kappa h}{a^2}\left(T_{m+1}^n - 2 T_m^n + T_{m-1}^n\right)}.$$

It may be easier to understand by writing the problem as a set of ODEs
$$\frac{\partial \phi_m}{\partial t} = \psi_m, \quad \text{and}\quad \frac{\partial \psi_m}{\partial t} = \frac{c^2}{a^2}\left(\phi_{m+1} - 2\phi_m + \phi_{m-1}\right)$$

and the discretization in time as:
$$
    \begin{bmatrix}
        \phi_m^{n+1} \\
        \psi_m^{n+1}
    \end{bmatrix}
    = 
    \begin{bmatrix}
        1 & +h \\
        -\frac{2hc^2}{a^2} & 1
    \end{bmatrix}
    \begin{bmatrix}
        \phi_m^{n} \\
        \psi_m^{n}
    \end{bmatrix}
    +
    \begin{bmatrix}
        0 \\
        \frac{c^2 h}{a^2}\left(\phi_{m+1}^n + \phi_{m-1}^n\right)
    \end{bmatrix}
$$

## Implicit FTCS Method

Evaluate the RHS of the above at time $t+h$ instead of $t$

* first do $h\to -h$ (from the current time step, compute the *previous* one):
    \begin{align*}
        \phi_m^{n-1} & = \phi_m^{n} - h\psi_m^{n},\\
        \psi_m^{n-1} & = \psi_m^{n} - h\frac{c^2}{a^2}\left(\phi_{m-1}^{n} + \phi_{m+1}^{n} - 2\phi_m^{n}\right),
    \end{align*}
* Then, $n \to n+1$ (one shift forward in time):
    \begin{align*}
        \phi_m^{n} & = \phi_m^{n+1} - h\psi_m^{n+1},\\
        \psi_m^{n} & = \psi_m^{n+1} - h\frac{c^2}{a^2}\left(\phi_{m-1}^{n+1} + \phi_{m+1}^{n+1} - 2\phi_m^{n+1}\right),
    \end{align*}
    or 
    $$
    \begin{bmatrix}
        \phi_m^n \\
        \psi_m^n
    \end{bmatrix}
    = 
    \begin{bmatrix}
        1 & -h \\
        +\frac{2hc^2}{a^2} & 1
    \end{bmatrix}
    \begin{bmatrix}
        \phi_m^{n+1} \\
        \psi_m^{n+1}
    \end{bmatrix}
    -
    \begin{bmatrix}
        0 \\
        \frac{c^2 h}{a^2}\left(\phi_{m+1}^{n+1} + \phi_{m-1}^{n+1}\right)
    \end{bmatrix}
    $$

"Implicit": we now have a set of simultaneous equations relating the values of $\phi,~\psi$ at $t$ to their values at $t+h$.

Why bother solving these simultaneous equations, rather than using an "explicit" expression for the values of $\phi,~\psi$ at $t+h$ given their values at $t$ ?

Because in certain cases, this is numerically stable while the explicit FTCS is not! (More about this later)

Note, it does often suffer from accuracy issues, where solutions decay to 0 over time.

## Crank-Nicolson

Average of explicit and implicit methods.

Explicit ('forward'):
$$ 
\begin{align}
\phi_m^{n+1} &= \phi_m^{n} + h\psi_m^{n}, & \psi_m^{n+1} = \psi_m^{n} + h\frac{c^2}{a^2}\left(\phi_{m-1}^{n} + \phi_{m+1}^{n} - 2\phi_m^{n}\right).
\end{align}
$$
Implicit ('backward'):
$$ 
\begin{align}
\phi_m^{n+1} - h\psi_m^{n+1} &= \phi_m^n, &\psi_m^{n} = \psi_m^{n+1} - h\frac{c^2}{a^2}\left(\phi_{m-1}^{n+1} + \phi_{m+1}^{n+1} - 2\phi_m^{n+1}\right).
\end{align}
$$
Crank-Nicholson (C-N):
$$ 
\phi_m^{n+1} - \frac{h}2\psi_m^{n+1} = \phi_m^{n} + \frac{h}2\psi_m^{n}
$$
$$ \psi_m^{n+1} - \frac{h}2\frac{c^2}{a^2}\left(\phi_{m-1}^{n+1} + \phi_{m+1}^{n+1} - 2\phi_m^{n+1}\right) = \psi_m^{n} + \frac{h}2\frac{c^2}{a^2}\left(\phi_{m-1}^{n} + \phi_{m+1}^{n} - 2\phi_m^{n}\right).
$$

C-N is 2nd-order accurate in time, while both explicit and implicit methods are 1st-order accurate. So, C-N often solves the 'decaying to 0' issues encountered with the implicit method.

# Hyperbolic PDEs

Explicit FTCS is always unstable. Use C-N, or spectral methods (next time)